In [1]:
import pandas as pd
from src.preprocessing import prepare_product_df
from src.matching import find_products_in_review
from src.llm import agent



df = pd.read_excel('data/zkusenosti.xlsx')
product_df = prepare_product_df(pd.read_excel("data/products.xlsx"))


c:\Users\Dominik\Documents\zkusenost-updater\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Dominik\Documents\zkusenost-updater\src\llm.py:15: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


In [2]:
from IPython.display import clear_output
import time
import json
import os

# File to store the last processed index
progress_file = "progress.txt"

# Load the last saved index if available
if os.path.exists(progress_file):
    with open(progress_file, "r") as f:
        try:
            ipos = int(f.read().strip())
        except ValueError:
            ipos = 0
else:
    ipos = 0  # Default start index

df_updated = df.copy()
last_successful_index = ipos  # Track the last successful iteration

for index, row in df_updated.iloc[ipos:].iterrows():
    clear_output(wait=True)
    print(f"Progress: {index}/{df_updated.shape[0]}")

    try:
        res = await agent({"zkusenost": row['zkusenost'], 'lng': row['lng'], 'nadpis': row['nadpis']})
        
        new_prod_ids = find_products_in_review(res['zkusenost'], product_df)['ID'].astype(str).tolist()
        existing_prod_ids = row.get('prodID', "").split(", ") if row.get('prodID') else []
        unique_prod_ids = list(dict.fromkeys(existing_prod_ids + new_prod_ids))

        res['prodID'] = ", ".join(unique_prod_ids)
        df_updated.loc[index, ['prodID', 'nadpis', 'tagy']] = [res['prodID'], res['nadpis'], res['tagy']]

        # Update progress tracking
        last_successful_index = index
        with open(progress_file, "w") as f:
            f.write(str(last_successful_index))

    except Exception as e:
        print(f"{e} - waiting")
        with open(progress_file, "w") as f:
            f.write(str(index))  # Save progress before sleeping
        df_updated.to_excel('data/products_updated.xlsx', index=False)  # Save on failure
        time.sleep(60)  # Wait before retrying

# Final save after loop completes successfully
print("Processing complete! Saving final version of the dataset.")
df_updated.to_excel('data/products_updated.xlsx', index=False)


Progress: 1/48018


CancelledError: 